# 신경망 모델 구성하기
- 신경망은 데이터에 대한 연산을 수행하는 계층(layer)과 모듈(module)로 구성되어 있다. 
- torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.
- 파이토치의 모든 모듈은 nn.Moudle의 하위 클래스이다.
- 신경망은 다른 모듈(계층;layer)로 구성된 모듈이다.
- 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

### FashionMNIST 데이터셋의 이미지들을 분류하는 신경망 구성

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
"""학습을 위한 장치 얻기"""
## GPU 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


-----

### 클래스 정의하기
- 신경망 모델을 nn.Module의 하위클래스로 정의하고, __init__에서 신경망 계층들을 초기화한다. 
- nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        ## flatten layer
        self.flatten = nn.Flatten()
        
        ## linear layer
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
"""NeuralNetwork의 인스턴스(instance)를 생성하고, 이를 device로 이동한 뒤 구조(structure)를 출력한다."""
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
"""
모델을 사용하기 위해 입력 데이터를 전달한다. 이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행한다.(model.forward()를 직접 호출하지 말기)

모델에 입력을 호출하면 각 target class에 대한 raw 예측값이 있는 10차원 텐서가 반환된다. raw 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.
"""

X = torch.rand(1, 28, 28, device = device)

## model(X)을 통해 raw 예측값 출력
logits = model(X)

## raw 예측값을 Softmax()에 넣기
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


--------

### 모델 계층(Layer)
- FashionMNIST 모델의 계층들을 살펴보겠다. 이를 위해 28*28 size의 이미지 3개로 구성된 minibatch를 가져와 신경망을 통과할 때 어떤 일이 발생하는지 보자.

In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


----

### nn.Flatten()
- nn.Flatten() 계층을 초기화하여 각 28*28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환

In [9]:
"""dim=0의 minibatch 차원은 그대로 유지됨"""
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


--------

### nn.Linear
- 선형 계층은 저장된 weight와 bias를 사용하여 입력에 선형 변환을 적용하는 모듈이다.

In [11]:
"""dim=0의 minibatch 차원은 그대로 유지됨"""
## Linear layer 하나만 만드릭
layer1 = nn.Linear(in_features = 28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


--------

### nn.ReLU
- 비선형 활성화는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만든다.
- 비선형성을 도입하여 신경망이 다양한 현상을 학습할 수 있도록 도움
- 이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1483,  0.1651,  0.3092, -0.1367,  0.1695,  0.9259,  0.4148, -0.0642,
          0.4373,  0.4728, -0.5555, -0.2597, -0.6533,  0.4282,  0.0091, -0.5497,
          0.1929,  0.0291, -0.3109, -0.3549],
        [ 0.1188, -0.1149, -0.0461, -0.2117,  0.1255,  0.8773,  0.4396, -0.0485,
         -0.0879,  0.3923, -0.6350, -0.3957, -0.4411,  0.3937,  0.4923, -0.3180,
          0.1492, -0.0120, -0.3423, -0.1487],
        [ 0.0724,  0.0730,  0.3313, -0.1869,  0.0412,  0.8972,  0.4337,  0.0970,
          0.1788,  0.1512, -0.5526, -0.3061, -0.5508,  0.4478,  0.1720, -0.5585,
          0.7487,  0.3660, -0.5037, -0.0148]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1483, 0.1651, 0.3092, 0.0000, 0.1695, 0.9259, 0.4148, 0.0000, 0.4373,
         0.4728, 0.0000, 0.0000, 0.0000, 0.4282, 0.0091, 0.0000, 0.1929, 0.0291,
         0.0000, 0.0000],
        [0.1188, 0.0000, 0.0000, 0.0000, 0.1255, 0.8773, 0.4396, 0.0000, 0.0000,
         0.3923, 0.0000, 0.0000, 0.0000, 0.3937, 0.492

--------

### nn.Sequential
- nn.Sequential은 순서를 갖는 모듈의 컨테이너이다. 
- 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다.
- 순차 컨테이너를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있다.

In [16]:
seq_modules = nn.Sequential(
flatten,
layer1,
nn.ReLU(),
nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits.shape

torch.Size([3, 10])

In [18]:
"""dim 매개변수는 값의 합이 1이 되는 차원을 의미한다."""
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0717, 0.1048, 0.1241, 0.1011, 0.1106, 0.1069, 0.0833, 0.0732, 0.1182,
         0.1061],
        [0.0691, 0.0982, 0.1495, 0.0983, 0.0987, 0.1012, 0.0884, 0.0712, 0.1187,
         0.1066],
        [0.0740, 0.1013, 0.1409, 0.0890, 0.0998, 0.1191, 0.0866, 0.0670, 0.1103,
         0.1120]], grad_fn=<SoftmaxBackward>)

-----

### 모델 매개변수
- 신경망 내부의 많은 계층들은 매개변수화 된다. 즉, 학습 중에 최적화되는 가중치와 편형과 연관지어진다. 
- nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다.

In [20]:
"""모델 구조"""
print(f"Model structure: {model}\n\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




In [22]:
"""각 layer 별로 weight와 bias가 번갈아가면서 나타난다."""
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0113,  0.0210,  0.0057,  ...,  0.0242, -0.0182,  0.0136],
        [ 0.0265, -0.0356,  0.0002,  ...,  0.0279, -0.0236,  0.0275]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0026, -0.0120], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0221,  0.0184, -0.0209,  ...,  0.0161, -0.0377, -0.0424],
        [ 0.0320,  0.0424, -0.0185,  ...,  0.0337, -0.0294, -0.0101]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0048,  0.0229], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0174,  0.0219, -0.0176,  ...,  0.0424, -0.0328,  0.0219],
        [ 0.0176, -0.0330, -0.0269,  ..